# Folosim modelu MobileNet pentru clasificarea maimutelor
## incarca modelul MobileNet


Înghețați toate straturile, cu excepția primilor 4, deoarece vom pregăti doar primele 4

In [2]:
from keras.applications import MobileNet

#MobileNet a fost proiectat sa lucreze cu dimensiunea imaginilor de 224 x 224 pixel
img_rows, img_cols = 224,224

#Reîncărcați modelul MobileNet fără straturile superioare sau FC
MobileNet = MobileNet(weights = 'imagenet', 
                      include_top= False,
                      input_shape = (img_rows, img_cols, 3)   
                         )
            
#Aici inghetam utimele 4 straturi
#În mod implicit, straturile sunt instruibile ca True
for layer in MobileNet.layers:
    layer.trainable = False

#printeaza straturile
for (i, layer) in enumerate(MobileNet.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)


0 InputLayer False
1 ZeroPadding2D False
2 Conv2D False
3 BatchNormalization False
4 ReLU False
5 DepthwiseConv2D False
6 BatchNormalization False
7 ReLU False
8 Conv2D False
9 BatchNormalization False
10 ReLU False
11 ZeroPadding2D False
12 DepthwiseConv2D False
13 BatchNormalization False
14 ReLU False
15 Conv2D False
16 BatchNormalization False
17 ReLU False
18 DepthwiseConv2D False
19 BatchNormalization False
20 ReLU False
21 Conv2D False
22 BatchNormalization False
23 ReLU False
24 ZeroPadding2D False
25 DepthwiseConv2D False
26 BatchNormalization False
27 ReLU False
28 Conv2D False
29 BatchNormalization False
30 ReLU False
31 DepthwiseConv2D False
32 BatchNormalization False
33 ReLU False
34 Conv2D False
35 BatchNormalization False
36 ReLU False
37 ZeroPadding2D False
38 DepthwiseConv2D False
39 BatchNormalization False
40 ReLU False
41 Conv2D False
42 BatchNormalization False
43 ReLU False
44 DepthwiseConv2D False
45 BatchNormalization False
46 ReLU False
47 Conv2D False
48 Batc

# Facem o functie care returneaza capul FC

In [6]:
def addTopModelMobileNet(bottom_model, num_classes):
    "" "creează partea superioară sau capul modelului care va fi plasat pe partea de sus a straturilor de jos" ""

    top_model = bottom_model.output#lum toate straturile pana la FC (86 ReLU)
    top_model = GlobalAveragePooling2D()(top_model)#de aici adugam straturile nostre prsonalizate 
    top_model = Dense(1024, activation='relu')(top_model)
    top_model = Dense(1024, activation='relu')(top_model)
    top_model = Dense(512, activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model
    #return full_model

# Adaugam capul FC inapoi in MobileNet

In [7]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D, Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization

#Seteaza class number la 3 (Tânăr, Mijloc, Bătrân)
num_classes = 10

FC_Head = addTopModelMobileNet(MobileNet, num_classes)

model = Model(inputs = MobileNet.input, outputs= FC_Head)
print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

# Incarcarea baza de date Monkey Breed

In [8]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = "c:/Users/Harum/Documents/15/monkey_breed/train"
validation_data_dir ="c:/Users/Harum/Documents/15/monkey_breed/validation"

#utiliza data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

#seteaza batch size (de obicei pe majoritatea sistemelor de nivel mediu, vom folosi 16-32)
batch_size = 32

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


# Instruieste modelul
        -Folosim checkpointing and early stopping

In [12]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("c:/Users/Harum/Documents/15/monkey_breed_mobileNet.h5",
                           monitor='val_loss',
                           mode='min',
                           save_best_only= True,
                           verbose=1
)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,#daca dupa n-1 adica 2 epochs consecutiv nu sa imbunatati nimic acesta model se va opri adica dupa terminarea celui deal treile epoch se va opri
                          verbose=1,
                          restore_best_weights= True
)

#punem call backs intro lista a callback
callbacks = [earlystop, checkpoint]

#folosim o rata de invatare foarte mica
model.compile(RMSprop(0.0001),'categorical_crossentropy', ['accuracy'])

#Introduce numarul de instruire si validare 
nb_train_samples = 1098
nb_validation_samples = 272

#parametri de instruire
epochs =10
batch_size= 32

#instruire model
history = model.fit_generator(
    train_generator,
    steps_per_epoch= nb_train_samples // batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data= validation_generator,
    validation_steps= nb_validation_samples // batch_size
)

Epoch 1/10
34/34 [==============================] - 31s 898ms/step - loss: 0.2346 - accuracy: 0.9231 - val_loss: 0.2513 - val_accuracy: 0.9258

Epoch 00001: val_loss improved from inf to 0.25127, saving model to c:/Users/Harum/Documents/15/monkey_breed_mobileNet.h5
Epoch 2/10
34/34 [==============================] - 32s 941ms/step - loss: 0.2010 - accuracy: 0.9334 - val_loss: 0.1351 - val_accuracy: 0.9500

Epoch 00002: val_loss improved from 0.25127 to 0.13506, saving model to c:/Users/Harum/Documents/15/monkey_breed_mobileNet.h5
Epoch 3/10
34/34 [==============================] - 31s 897ms/step - loss: 0.1951 - accuracy: 0.9268 - val_loss: 0.3944 - val_accuracy: 0.9500

Epoch 00003: val_loss did not improve from 0.13506
Epoch 4/10
34/34 [==============================] - 30s 889ms/step - loss: 0.1843 - accuracy: 0.9311 - val_loss: 0.1528 - val_accuracy: 0.9458

Epoch 00004: val_loss did not improve from 0.13506
Epoch 5/10
34/34 [==============================] - 30s 886ms/step - loss:

# Incarca clasificatorul

In [13]:
from keras.models import load_model

classifier = load_model("c:/Users/Harum/Documents/15/monkey_breed_mobileNet.h5")

In [14]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

monkey_breeds_dict = {"[0]": "mantled_howler ", 
                      "[1]": "patas_monkey",
                      "[2]": "bald_uakari",
                      "[3]": "japanese_macaque",
                      "[4]": "pygmy_marmoset ",
                      "[5]": "white_headed_capuchin",
                      "[6]": "silvery_marmoset",
                      "[7]": "common_squirrel_monkey",
                      "[8]": "black_headed_night_monkey",
                      "[9]": "nilgiri_langur"}

monkey_breeds_dict_n = {"n0": "mantled_howler ", 
                      "n1": "patas_monkey",
                      "n2": "bald_uakari",
                      "n3": "japanese_macaque",
                      "n4": "pygmy_marmoset ",
                      "n5": "white_headed_capuchin",
                      "n6": "silvery_marmoset",
                      "n7": "common_squirrel_monkey",
                      "n8": "black_headed_night_monkey",
                      "n9": "nilgiri_langur"}

def draw_test(name, pred, im):
    monkey = monkey_breeds_dict[str(pred)]
    BLACK = [0, 0, 0]
    expanded_image = cv2.copyMakeBorder(im, 80,0,0,100, cv2.BORDER_CONSTANT, value=BLACK)
    cv2.putText(expanded_image, monkey, (20,60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    """funcția încarcă imagini aleatoare dintr-un folder aleatoriu în calea noastră de testare"""
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0, len(folders))
    path_class = folders[random_directory]
    print("Class - "+ monkey_breeds_dict_n[str(path_class)])
    file_path = path +path_class
    file_names = [ f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0, len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+ "/"+ image_name)

for i in range(0,10):
    input_im = getRandomImage("c:/Users/Harum/Documents/15/monkey_breed/validation/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)

    input_im = cv2.resize(input_im, (224,224), interpolation= cv2.INTER_LINEAR)
    input_im =input_im / 255.
    input_im = input_im.reshape(1, 224,224,3)

    #Predictia
    res = np.argmax(classifier.predict(input_im, 1, verbose=0), axis=1)

    #Arata imaginea cu clasa prezisa
    draw_test("Prediction", res, input_original)
    cv2.waitKey(0)

cv2.destroyAllWindows()

Class - silvery_marmoset
Class - silvery_marmoset
Class - black_headed_night_monkey
Class - white_headed_capuchin
Class - mantled_howler 
Class - bald_uakari
Class - nilgiri_langur
Class - black_headed_night_monkey
Class - black_headed_night_monkey
Class - white_headed_capuchin
